# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [113]:
# Import Modules
from afl_feature_creation import prepare_afl_features
import afl_data_cleaning
import afl_feature_creation
import afl_modelling
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [114]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [115]:
next_week_odds

,home_team,away_team,odds,odds_away
0,Sydney,Geelong,1.51,2.98
1,Richmond,Adelaide,1.21,5.80
2,Brisbane,Carlton,1.44,3.10
3,Port Adelaide,St Kilda,1.14,8.20
4,Western Bulldogs,Hawthorn,3.30,1.44
5,Melbourne,Fremantle,1.15,7.60
6,North Melbourne,Gold Coast,1.11,10.50
7,Essendon,Collingwood,2.58,1.63
8,West Coast,GWS,1.66,2.50


In [116]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
211,2018-08-24,2018,212,NaN,Brisbane,West Coast,Gabba
212,2018-08-24,2018,213,NaN,Carlton,St Kilda,Etihad Stadium
213,2018-08-24,2018,214,NaN,Richmond,Melbourne,MCG
214,2018-08-24,2018,215,NaN,Port Adelaide,Essendon,Adelaide Oval
215,2018-08-24,2018,216,NaN,Fremantle,Collingwood,Optus Stadium


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [117]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['Game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids

# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['Game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season', 'Season.Game', 'Round'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])

    # Split the DataFrame onto two rows for each game
    h_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds']]
    h_df['Team'] = h_df['home_team']
    h_df['Opposition'] = h_df['away_team']
    h_df['Home?'] = 1
    a_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds_away']].rename(columns={'odds_away': 'odds'})
    a_df['Team'] = a_df['away_team']
    a_df['Opposition'] = a_df['home_team']
    a_df['Home?'] = 0
    next_week = a_df.append(h_df).sort_values(by='Game')
    return next_week

In [118]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df.head()

,Date,Game,home_team,away_team,odds,Team,Opposition,Home?
0,2018-07-06,15319,Richmond,Adelaide,5.80,Adelaide,Richmond,0
0,2018-07-06,15319,Richmond,Adelaide,1.21,Richmond,Adelaide,1
1,2018-07-07,15320,Brisbane,Carlton,3.10,Carlton,Brisbane,0
1,2018-07-07,15320,Brisbane,Carlton,1.44,Brisbane,Carlton,1
2,2018-07-07,15321,Port Adelaide,St Kilda,8.20,St Kilda,Port Adelaide,0


### Create Each Feature
Now let's append next week's DataFrame to our afl_data DataFrame and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb).

In [119]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)
afl_data = afl_data[ordered_cols]

# Create disposal efficiency column
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create rolling averages for our statistics
cols_indx_start = afl_data.columns.get_loc("GA")
afl_avgs = afl_feature_creation.create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

# Create form between teams feature
afl_avgs = afl_feature_creation.form_between_teams(afl_avgs, 6)

# Apply elos
elos, probs, elo_dict = afl_feature_creation.elo_applier(afl_data, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).str[0]
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).str[1]

# Get elos for next week
afl_avgs.loc[afl_avgs['home_elo'].isna(), 'home_elo'] = afl_avgs[afl_avgs['home_elo'].isna()]['home_team'].map(elo_dict)
afl_avgs.loc[afl_avgs['away_elo'].isna(), 'away_elo'] = afl_avgs[afl_avgs['away_elo'].isna()]['away_team'].map(elo_dict)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs = afl_feature_creation.map_elos(afl_avgs)
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

# Create average elo of opponents beaten/lost to feature
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

# Create regular margin rolling average
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Margin'])
afl_avgs.tail()

,Team,odds,Date,home_team,away_team,Behinds,Game,Goals,Home?,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,form_over_opposition_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,average_elo_opponents_beaten_6,average_elo_opponents_lost_6,Margin_ave_6
2961,North Melbourne,1.11,2018-07-08,North Melbourne,Gold Coast,NaN,15324,NaN,1,Gold Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.333333,151.333333,212.666667,257.166667,12.000000,10.833333,43.666667,5.666667,190.833333,169.333333,360.166667,74.000000,13.500000,8.333333,64.666667,43.333333,51.833333,38.166667,56.500000,34.166667,25.333333,21.000000,1486.833333,1631.666667,13.833333,24.333333,87.333333,5505.333333,72.333333,72.166667,12.166667,0.712037,2,1472.379995,1285.335813,1472.379995,1285.335813,11.671217,1506.090681,1507.511006,13.333333
2962,Collingwood,1.63,2018-07-08,Essendon,Collingwood,NaN,15325,NaN,0,Essendon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.666667,143.500000,280.500000,319.333333,13.000000,13.166667,50.500000,9.666667,230.166667,192.166667,422.333333,111.500000,14.166667,9.666667,65.000000,47.500000,56.333333,37.333333,52.500000,32.833333,22.666667,18.333333,1779.166667,1762.500000,11.833333,25.500000,117.000000,5832.333333,68.666667,70.166667,11.666667,0.755293,2,1478.524824,1528.013453,1528.013453,1478.524824,25.881265,1396.428752,1598.556956,27.500000
2963,Essendon,2.58,2018-07-08,Essendon,Collingwood,NaN,15325,NaN,1,Collingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.833333,146.000000,221.666667,263.666667,10.000000,10.833333,45.500000,11.500000,224.666667,145.666667,370.333333,97.000000,11.000000,8.833333,77.833333,44.166667,54.833333,40.666667,59.166667,40.166667,19.666667,24.000000,1634.333333,1696.500000,11.166667,29.500000,87.666667,5818.166667,70.833333,70.666667,12.666667,0.710224,4,1478.524824,1528.013453,1478.524824,1528.013453,6.486989,1558.995964,1486.833099,5.833333
2964,GWS,2.50,2018-07-08,West Coast,GWS,NaN,15326,NaN,0,West Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.500000,157.666667,220.000000,269.166667,11.333333,10.166667,63.666667,7.666667,211.333333,165.666667,377.000000,75.166667,12.500000,10.666667,72.166667,40.000000,56.666667,41.833333,59.166667,37.666667,18.666667,23.000000,1554.833333,1692.333333,12.000000,29.833333,103.166667,6027.333333,69.333333,74.666667,12.000000,0.713936,3,1618.106496,1568.458243,1568.458243,1618.106496,5.249531,1384.518069,1564.135373,8.000000
2965,West Coast,1.66,2018-07-08,West Coast,GWS,NaN,15326,NaN,1,GWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,142.666667,205.333333,253.166667,14.333333,11.000000,50.166667,3.166667,219.333333,128.666667,348.000000,100.000000,12.000000,8.833333,60.833333,45.500000,52.000000,36.166667,50.500000,34.833333,24.166667,17.500000,1556.666667,1641.833333,12.166667,24.000000,87.666667,5607.000000,69.666667,70.000000,10.000000,0.727897,3,1618.106496,1568.458243,1618.106496,1568.458243,9.452812,1533.683281,1580.099883,9.500000


Our DataFrame looks great! Although there are some NaNs, these are only in columns like 'Goals' which obviously don't have a value yet as the game hasn't been played.

Now we need to get our DataFrame on one line so that each row corresponds to one footy game. Let's use our previously defined function for this. Let's also drop the columns with NaNs which we don't need.

In [120]:
# Get each footy match on individual rows and drop irrelevant columns
one_line = afl_modelling.get_df_on_one_line(afl_avgs)

# Drop duplicate columns and unnecessary columns
cols_to_drop = ['Opposition Behinds', 'Goals', 'Behinds', 'Opposition Goals', 'Opposition Points', 'Points', 'Round', 'Venue', 'Season', 'Status',
               'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'elo', 'elo_Opp', 'Behinds_away',
               'Goals_away', 'home_elo_away', 'away_elo_away', 'elo_away', 'elo_Opp_away']
one_line = one_line.drop(columns=cols_to_drop)
# Drop all columns where home_elo or away_elo == 1500 exactly, as this is the first game played
one_line = one_line[(one_line['home_elo'] != 1500) & (one_line['away_elo'] != 1500)]

# Drop Na rows from calculating moving averages
one_line = one_line.dropna(axis=0)

Now let's create our differential_df and then filter out DataFrame to only include this weekend's game, based on the Game IDs we created earlier

In [121]:
diff_df = afl_modelling.get_diff_df(one_line)
diff_df = diff_df.drop(columns=['odds', 'odds_away']).select_dtypes(include=[np.number])
prediction_feature_set = diff_df[diff_df['Game'].isin(game_ids_next_round)].dropna(axis=1)

In [122]:
prediction_feature_set

,Game,home_elo,away_elo,CCL_ave_6_away,SCL_ave_6_away,SI_ave_6_away,MG_ave_6_away,TO_ave_6_away,ITC_ave_6_away,T5_ave_6_away,GA_ave_6_diff,CP_ave_6_diff,UP_ave_6_diff,ED_ave_6_diff,CM_ave_6_diff,MI5_ave_6_diff,One.Percenters_ave_6_diff,BO_ave_6_diff,K_ave_6_diff,HB_ave_6_diff,D_ave_6_diff,M_ave_6_diff,G_ave_6_diff,B_ave_6_diff,T_ave_6_diff,HO_ave_6_diff,I50_ave_6_diff,CL_ave_6_diff,CG_ave_6_diff,R50_ave_6_diff,FF_ave_6_diff,FA_ave_6_diff,AF_ave_6_diff,SC_ave_6_diff,disposal_efficiency_ave_6_diff,form_over_opposition_6_diff,Adj_elo_ave_margin_ave_6_diff,average_elo_opponents_beaten_6_diff,average_elo_opponents_lost_6_diff,Margin_ave_6_diff,implied_odds_prob,implied_odds_prob_away
1475,15319,1609.677631,1578.060630,10.000000,28.333333,67.000000,5610.500000,77.833333,75.500000,10.166667,2.166667,-9.500000,6.000000,5.500000,2.666667,5.000000,7.666667,1.500000,-9.666667,7.666667,-2.000000,-0.833333,3.500000,1.833333,-4.833333,-12.500000,9.500000,-4.833333,10.666667,0.833333,-1.500000,4.333333,-39.666667,103.666667,0.019319,-2,30.653344,19.348896,86.567317,33.333333,0.826446,0.172414
1476,15320,1233.037199,1280.164173,11.333333,24.500000,68.500000,4728.500000,69.500000,63.333333,10.500000,3.833333,5.500000,23.166667,33.166667,1.166667,4.166667,8.666667,-1.500000,7.166667,21.333333,28.500000,1.166667,3.666667,2.166667,-6.500000,19.333333,9.833333,1.333333,-5.833333,3.166667,-2.000000,-1.666667,88.166667,54.000000,0.034788,4,30.136248,-99.516290,-51.773326,25.333333,0.694444,0.322581
1477,15321,1589.121018,1400.692014,14.500000,21.500000,90.666667,5419.833333,67.500000,64.666667,10.333333,0.833333,24.166667,-51.666667,-36.500000,4.500000,1.666667,20.833333,-2.333333,11.833333,-31.500000,-19.666667,-10.333333,2.166667,1.500000,15.833333,21.833333,4.000000,8.500000,5.666667,8.666667,3.833333,3.833333,33.500000,151.166667,-0.057608,6,50.496897,63.897788,28.766582,48.500000,0.877193,0.121951
1478,15322,1446.605223,1606.319440,9.333333,24.333333,98.666667,5445.166667,68.333333,71.166667,12.166667,-1.000000,17.000000,-23.000000,-15.666667,0.000000,-1.333333,2.666667,0.166667,-21.000000,19.666667,-1.333333,-28.333333,-1.500000,-1.166667,2.500000,-10.166667,-4.333333,2.500000,0.166667,11.500000,0.333333,0.333333,-119.666667,-133.500000,-0.040535,-4,-35.338251,-125.984666,50.109171,-33.166667,0.303030,0.694444
1479,15323,1494.602682,1449.483864,13.833333,22.000000,91.833333,5172.333333,71.833333,68.333333,8.000000,5.833333,24.000000,5.166667,27.333333,3.500000,4.166667,-5.000000,0.500000,5.833333,29.166667,35.000000,-8.833333,7.000000,2.000000,5.166667,10.166667,15.333333,3.333333,0.500000,-5.000000,4.000000,2.666667,120.166667,122.333333,0.000084,-4,54.828014,0.515970,7.884760,54.000000,0.869565,0.131579
1480,15324,1472.379995,1285.335813,9.000000,25.333333,59.333333,5252.833333,74.666667,69.000000,10.666667,5.000000,15.500000,40.666667,50.333333,3.333333,4.666667,-6.166667,1.833333,-2.333333,52.666667,50.333333,-1.833333,6.666667,0.666667,-4.000000,-6.500000,9.333333,3.833333,-2.833333,-12.166667,1.166667,-2.666667,120.166667,204.666667,0.043227,-2,83.642606,52.040887,-37.488616,72.833333,0.900901,0.095238
1481,15325,1478.524824,1528.013453,11.833333,25.500000,117.000000,5832.333333,68.666667,70.166667,11.666667,-3.833333,2.500000,-58.833333,-55.666667,-3.000000,-2.333333,-5.000000,1.833333,-5.500000,-46.500000,-52.000000,-14.500000,-3.166667,-0.833333,12.833333,-3.333333,-1.500000,3.333333,6.666667,7.333333,-3.000000,5.666667,-144.833333,-66.000000,-0.045069,2,-19.394276,162.567212,-111.723857,-21.666667,0.387597,0.613497
1482,15326,1618.106496,1568.458243,12.000000,29.833333,103.166667,6027.333333,69.333333,74.666667,12.000000,-0.500000,-15.000000,-14.666667,-16.000000,3.000000,0.833333,-13.500000,-4.500000,8.000000,-37.000000,-29.000000,24.833333,-0.500000,-1.833333,-11.333333,5.500000,-4.666667,-5.666667,-8.666667,-2.833333,5.500000,-5.500000,1.833333,-50.500000,0.013961,0,4.203282,149.165213,15.964510,1.500000,0.602410,0.400000


Great! We now have our feature DataFrame for this weekend's games. Let's now model this to create our predictions

## Creating Our Predictions
To create our predictions we will use similar code to what we used in our AFL Modelling Tutorial. We will need to train our model on all the data we have (up until last week's games), and then use our trained model to predict this week. Again, we will use Stacking with XGB, like we did in our AFL Modelling Tutorial.

In [123]:
afl = prepare_afl_features(window=6, k_factor=24).dropna().sort_values(by='Game')
afl = afl_modelling.get_df_on_one_line(afl)

# Drop columns which leak data and which we don't need
dropped_cols = ['Behinds', 'Goals', 'Opposition Behinds', 'Opposition Goals', 'Opposition Points', 'Points',
               'elo', 'elo_Opp', 'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'home_win_away',
               'home_elo_away', 'away_elo_away']
afl = afl.drop(columns=dropped_cols)
# Get a differential DataFrame - subtracting the away features from the home features
diff_df = afl_modelling.get_diff_df(afl)

In [124]:
# Hard code estimators from our last tutorial
all_estimators = [
    LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
       penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
       verbose=0, warm_start=False),
    
    RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
     max_iter=None, normalize=False, random_state=None, solver='auto',
     tol=0.001),
    
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
         max_depth=15, max_features='sqrt', max_leaf_nodes=None,
         min_impurity_decrease=0.0, min_impurity_split=None,
         min_samples_leaf=1, min_samples_split=10,
         min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=-1,
         oob_score=False, random_state=5, verbose=0, warm_start=False),
    
    GaussianNB(priors=None),
    
    LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
           solver='svd', store_covariance=False, tol=0.0001)
]

# Hard code best cols
best_cols = ['home_elo',
'away_elo',
'GA_ave_6_diff',
'CP_ave_6_diff',
'UP_ave_6_diff',
'ED_ave_6_diff',
'CM_ave_6_diff',
'MI5_ave_6_diff',
'One.Percenters_ave_6_diff',
'BO_ave_6_diff',
'HB_ave_6_diff',
'M_ave_6_diff',
'G_ave_6_diff',
'T_ave_6_diff',
'HO_ave_6_diff',
'I50_ave_6_diff',
'CL_ave_6_diff',
'CG_ave_6_diff',
'R50_ave_6_diff',
'FF_ave_6_diff',
'FA_ave_6_diff',
'AF_ave_6_diff',
'SC_ave_6_diff',
'disposal_efficiency_ave_6_diff',
'R50_efficiency_ave_6_diff',
'I50_efficiency_ave_6_diff',
'Adj_elo_ave_margin_ave_6_diff',
'average_elo_opponents_beaten_6_diff',
'average_elo_opponents_lost_6_diff',
'Margin_ave_6_diff',
'implied_odds_prob',
'implied_odds_prob_away']

In [125]:
# Create our train sets
X = diff_df.drop(columns=['home_win']).select_dtypes(include=[np.number])
y = diff_df['home_win']

features = prediction_feature_set.columns

# Predict Next Round
preds_next_round = afl_modelling.implement_xgb_stacking(X[features].drop(columns='Game'), y, 
                                                        prediction_feature_set.drop(columns='Game'), all_estimators)

C:\Users\wardj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now that we have our predictions, the final step is to put our predictions into a DataFrame so they're easy on the eye, and our predictions correspond to actual home and away teams rather than Game IDs. This is the final step.

In [126]:
preds_df = pd.DataFrame({
    "Game": prediction_feature_set['Game'],
    "Prediction (home_win)": preds_next_round
})

prediction_feature_set['home_odds'] = 1 / prediction_feature_set['implied_odds_prob']
prediction_feature_set['away_odds'] = 1 / prediction_feature_set['implied_odds_prob_away']

final_preds_df = pd.merge(preds_df, next_week_df[['home_team', 'away_team', 'Game']], on='Game').drop_duplicates()
final_preds_df = pd.merge(final_preds_df, prediction_feature_set[['home_odds', 'away_odds', 'home_elo', 'away_elo', 'Game']], on='Game')
final_preds_df['Predicted Winner'] = final_preds_df.apply(lambda x: x['home_team'] if x['Prediction (home_win)'] == 1 else x['away_team'], axis=1)

In [127]:
final_preds_df[['Game', 'home_team', 'away_team', 'Predicted Winner', 'home_odds', 'away_odds', 'home_elo', 'away_elo']]

,Game,home_team,away_team,Predicted Winner,home_odds,away_odds,home_elo,away_elo
0,15319,Richmond,Adelaide,Richmond,1.21,5.80,1609.677631,1578.060630
1,15320,Brisbane,Carlton,Brisbane,1.44,3.10,1233.037199,1280.164173
2,15321,Port Adelaide,St Kilda,Port Adelaide,1.14,8.20,1589.121018,1400.692014
3,15322,Western Bulldogs,Hawthorn,Hawthorn,3.30,1.44,1446.605223,1606.319440
4,15323,Melbourne,Fremantle,Melbourne,1.15,7.60,1494.602682,1449.483864
5,15324,North Melbourne,Gold Coast,North Melbourne,1.11,10.50,1472.379995,1285.335813
6,15325,Essendon,Collingwood,Collingwood,2.58,1.63,1478.524824,1528.013453
7,15326,West Coast,GWS,West Coast,1.66,2.50,1618.106496,1568.458243


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!